In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize

%matplotlib inline

In [2]:
INPUT_SIZE = (224, 224)

# Training

In [3]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")

Torch: 1.9.0+cu102


In [4]:
# Training settings
batch_size = 32# 32 #16 #8 #
epochs = 40
lr = 3e-5
gamma = 0.7
seed = 42
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [5]:
IMG_SIZE=224 # 300 # 80 #

train_transforms = transforms.Compose(
    [
        #transforms.Resize((IMG_SIZE_ORIG,IMG_SIZE_ORIG)),
        #transforms.RandomResizedCrop(IMG_SIZE),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

In [6]:
train_dir_1 = r'C:\Users\HP\SLR\crossVal\Data\fold_1_c\train_files'
test_dir_1 = r'C:\Users\HP\SLR\crossVal\Data\fold_1_c\test_files'

train_dir_2 = r'C:\Users\HP\SLR\crossVal\Data\fold_2_c\train_files'
test_dir_2 = r'C:\Users\HP\SLR\crossVal\Data\fold_2_c\test_files'

train_dir_3 = r'C:\Users\HP\SLR\crossVal\Data\fold_3_c\train_files'
test_dir_3 = r'C:\Users\HP\SLR\crossVal\Data\fold_3_c\test_files'

train_dir_4 = r'C:\Users\HP\SLR\crossVal\Data\fold_4_c\train_files'
test_dir_4 = r'C:\Users\HP\SLR\crossVal\Data\fold_4_c\test_files'

train_dir_5 = r'C:\Users\HP\SLR\crossVal\Data\fold_5_c\train_files'
test_dir_5 = r'C:\Users\HP\SLR\crossVal\Data\fold_5_c\test_files'


kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

train_dataset_1 = datasets.ImageFolder(root=train_dir_1, transform=train_transforms)
test_dataset_1 = datasets.ImageFolder(root=test_dir_1, transform=test_transforms)


train_dataset_2 = datasets.ImageFolder(root=train_dir_2, transform=train_transforms)
test_dataset_2 = datasets.ImageFolder(root=test_dir_2, transform=test_transforms)

train_dataset_3 = datasets.ImageFolder(root=train_dir_3, transform=train_transforms)
test_dataset_3 = datasets.ImageFolder(root=test_dir_3, transform=test_transforms) 

train_dataset_4 = datasets.ImageFolder(root=train_dir_4, transform=train_transforms)
test_dataset_4 = datasets.ImageFolder(root=test_dir_4, transform=test_transforms)

train_dataset_5 = datasets.ImageFolder(root=train_dir_5, transform=train_transforms)
test_dataset_5 = datasets.ImageFolder(root=test_dir_5, transform=test_transforms)

training_set = [train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4, train_dataset_5]
testing_set = [test_dataset_1, test_dataset_2, test_dataset_3, test_dataset_4, test_dataset_5]
test_dirs = [test_dir_1, test_dir_2, test_dir_3, test_dir_4, test_dir_5]

print(len(training_set), len(testing_set), len(test_dirs))

5 5 5


# Training

In [7]:
from sam import SAM
import copy
def train(model,n_epochs=epochs, learningrate=lr, use_sam=False):
    # optimizer
    
    if use_sam:
        optimizer = SAM(filter(lambda p: p.requires_grad, model.parameters()), optim.Adam, lr=learningrate)
    else:
        optimizer=optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)
    # scheduler
    #scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()

        for data, label in tqdm(train_loader):
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            loss = criterion(output, label)

            if use_sam:
                #optimizer.zero_grad()
                loss.backward()
                optimizer.first_step(zero_grad=True)

                # second forward-backward pass
                output = model(data)
                loss = criterion(output, label)
                loss.backward()
                optimizer.second_step(zero_grad=True)
            else:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in test_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc / len(test_loader)
                epoch_val_loss += val_loss / len(test_loader)

        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        #scheduler.step()

    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in test_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc / len(test_loader)
                epoch_val_loss += val_loss / len(test_loader)

        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")

In [8]:
#adapted from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
def set_parameter_requires_grad(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [9]:
from torchvision.models import resnet101,mobilenet_v2
import sys
import timm
sys.path.insert(0, 'timm==0.4.5')
#import timm #gets version 2
print(f"Timm: {timm.__version__}")

Timm: 0.4.5


In [ ]:
k = 1
num_classes = 7

for test_dir, train_dataset, test_dataset in zip(test_dirs, training_set, testing_set):
    
    print("Training model number: " + str(k))
    
    # load the dataset
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
    test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)
    
    (unique, counts) = np.unique(train_dataset.targets, return_counts=True)
    cw=1/counts
    cw/=cw.min()
    class_weights = {i:cwi for i,cwi in zip(unique,cw)}
    #print(counts, class_weights.values())
    
    # loss function
    weights = torch.FloatTensor(list(class_weights.values())).cuda()
    criterion = nn.CrossEntropyLoss(weight=weights)
    #criterion = nn.CrossEntropyLoss()
    
    # loading the pre-trained model
    model=timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
    model.classifier=torch.nn.Identity()
    model = torch.load(r'C:/Users/HP/SLR/models/affectnet_emotions/enet_b0_7.pt') #_new
    
    model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=num_classes)) #1792 #1280 #1536
    #model.head.fc=nn.Linear(in_features=3072, out_features=num_classes)
    #model.head=nn.Sequential(nn.Linear(in_features=768, out_features=num_classes))
    model=model.to(device)
    #print(model)
    
    if False:
        set_parameter_requires_grad(model.features, requires_grad=False)
    else:
        set_parameter_requires_grad(model, requires_grad=False)
        set_parameter_requires_grad(model.classifier, requires_grad=True)
        #set_parameter_requires_grad(model.head, requires_grad=True)
    train(model,3,0.001,use_sam=True)
    #Best acc:0.48875007033348083
    #7: Best acc:0.558712363243103
    
    if False:
        set_parameter_requires_grad(model.features, requires_grad=True)
    else:
        set_parameter_requires_grad(model, requires_grad=True)
    
    train(model,6,1e-4,use_sam=True)
    
    if k==1:
        PATH='C:/Users/HP/SLR/crossVal/EfficientNet/Model_1_pt'
        model_name='Model_1_pt'
    elif k==2:
        PATH='C:/Users/HP/SLR/crossVal/EfficientNet/Model_2_pt'
        model_name='Model_2_pt'
    elif k==3:
        PATH='C:/Users/HP/SLR/crossVal/EfficientNet/Model_3_pt'
        model_name='Model_3_pt'
    elif k==4:
        PATH='C:/Users/HP/SLR/crossVal/EfficientNet/Model_4_pt'
        model_name='Model_4_pt'
    elif k==5:
        PATH='C:/Users/HP/SLR/crossVal/EfficientNet/Model_5_pt'
        model_name='Model_5_pt'

    # Save
    torch.save(model, PATH)
    
    # Evaluation
    class_to_idx=train_dataset.class_to_idx
    idx_to_class={idx:cls for cls,idx in class_to_idx.items()}
    
    y_val,y_scores_val=[],[]
    model.eval()
    for class_name in tqdm(os.listdir(test_dir)):
        if class_name in class_to_idx:
            class_dir=os.path.join(test_dir,class_name)
            y=class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                filepath=os.path.join(class_dir,img_name)
                img = Image.open(filepath)
                img_tensor = test_transforms(img)
                img_tensor.unsqueeze_(0)
                scores = model(img_tensor.to(device))
                scores=scores[0].data.cpu().numpy()
                #print(scores.shape)
                y_scores_val.append(scores)
                y_val.append(y)

    y_scores_val=np.array(y_scores_val)
    y_val=np.array(y_val)
    #print(y_scores_val.shape,y_val.shape)
    
    # Accuracies    
    y_pred=np.argmax(y_scores_val,axis=1)
    acc=100.0*(y_val==y_pred).sum()/len(y_val)
    print(acc)
    y_train=np.array(train_dataset.targets)
    for i in range(y_scores_val.shape[1]):
        _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
        print('%s %d/%d acc: %f' %(idx_to_class[i],(y_train==i).sum(),(y_val==i).sum(),100*_val_acc))
        
    # Classification Report 
    from sklearn.metrics import classification_report
    target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
    print(classification_report(y_val, y_pred, target_names=target_names))

    print('Model number: ' + str(k) + ' compiled!')
    
    k = k + 1

Training model number: 1


  0%|          | 0/64 [00:00<?, ?it/s]